# Part 1

In [42]:
import pandas as pd
import numpy as np
import requests
import geocoder

In [43]:
#get data
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [44]:
#load into pandas, first table is the one we want. Also we set first row as header
loaded_table = pd.read_html(html.text,header=0)[0]
loaded_table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Now we have to remove rows where Borough is not assigned:

In [45]:
#this filters out all rows, where Borough has value 'Not assigned'
loaded_table = loaded_table[loaded_table['Borough'] != 'Not assigned']
loaded_table.head(7)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue


For cells which have _Not assigned_ neighbourhood use borough name as neighbourhood name:

In [46]:
#define function applied to each row. This will change Neighbourhood in case its value is 'Not assigned'
def change(row):
    postcode,borough,neighbourhood = row
    if neighbourhood == 'Not assigned':
        neighbourhood = borough
    return pd.Series([postcode,borough,neighbourhood])

#apply the function to each row
loaded_table = loaded_table.apply(func=change, axis=1, result_type='broadcast')
loaded_table.head(7)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue


Now we need to regroup rows in our table, so that all neighbourhoods with same postcode (and same borough) are on one row:

In [47]:
#grouping by postcode --> that means each row will represent unique postcode. Then aggregating by unique neighbourhoods
final_table = loaded_table.groupby('Postcode').agg(lambda x: ', '.join(set(x))).reset_index()
final_table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now we will print shape of final output

In [48]:
print('shape of final table is: '+str(final_table.shape))

shape of final table is: (103, 3)


# Part 2
We will use geospatial data from https://cocl.us/Geospatial_data to get latitude and longtitude of each post code.

In [49]:
geospat = pd.read_csv('https://cocl.us/Geospatial_data')

In [50]:
#create function which requests coordinates for each postcode from ArcGis
def add_lat_long(row):
    postcode,borough,neigh = row
    latlong = geospat[geospat['Postal Code'] == postcode].iloc[0].to_dict()
    return pd.Series([latlong['Latitude'],latlong['Longitude']])

#get coordinates of each postcode, save it to coordinates dataframe
coordinates = final_table.apply(axis=1, func=add_lat_long, result_type=None)
coordinates.columns = ['latitude','longtitude']
coordinates.head()

,latitude,longtitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [51]:
#now concatenate columns from coordinates dataframe with columns from original dataframe
data_with_coordinates = pd.concat([final_table,coordinates],axis=1)
data_with_coordinates.head()

,Postcode,Borough,Neighbourhood,latitude,longtitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
